<a href="https://colab.research.google.com/github/mb8655/Python/blob/main/Regular_Expressions_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Regular Expressions
Regular expressions (regexes or re’s) constitute an extremely powerful, flexible and concise language for matching elements in text ranging from a few characters to complex patterns. While mastering the syntax of the regular expression language does require climbing a learning curve, this learning curve is not particularly steep, and a newcomer can find herself performing useful tasks with regular expressions almost immediately. Efforts spent learning regular expressions quickly pay off--tasks that are well suited for regular expressions abound. Indeed, regular expressions are one of the most useful computer skills, and an absolutely critical tool for data scientists.

This document will present basic regular expression syntax and cover common use cases for regular expressions.

In [1]:
# The code below is written in Python to replicate the
# behavior of grep, the UNIX utility
# We will examine the details of how the code works in a subsequent notebook.
# For now, just execute the code, and use the function
# grep(regex_expression, name_list) as-is

import re

def printMatches(text, regex_expression):
  BACKGROUND_YELLOW = '\x1b[43m'
  COLOR_RESET  = "\x1b[0m"
  regex= re.compile(regex_expression)
  matches = regex.finditer(text)
  for m in matches:
    highlighted  = text[:m.start()] # the string before the regex match
    highlighted += BACKGROUND_YELLOW + text[m.start():m.end()] + COLOR_RESET
    highlighted += text[m.end():] # the string after the regex match
    print(highlighted)

def grep(regex_expression, name_list):
  for line in name_list:
    printMatches(line, regex_expression)

## NYC Restaurant Names Data
In the notebook, we will demonstrating the various regular expressions using the set of restaurant names from /data/uniquenames.txt.

In [2]:
!pip install -U -q PyMySQL sqlalchemy

from sqlalchemy import create_engine
import pandas as pd

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org',
    user = 'student',
    password = 'dwdstudent2025',
    db = 'doh_restaurants',
    encoding = 'utf8mb4')

with create_engine(conn_string).connect() as mysql_conn:
  # This query returns back the restaurants in the DOH database
  sql = 'SELECT DISTINCT UPPER(DBA) AS DBA FROM restaurants WHERE DBA IS NOT NULL'
  uniquenames = pd.read_sql(sql, con=mysql_conn)
  uniquenames = uniquenames.DBA.values

print(f"Found {len(uniquenames)} unique restaurants names")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.2 MB/s eta 0:00:00
Found 15721 unique restaurants names


In [3]:
#Let's take a peek at the entries of the uniquenames list:

uniquenames[:5]

array(['MORRIS PARK BAKE SHOP', "WENDY'S",
       'DJ REYNOLDS PUB AND RESTAURANT', 'RIVIERA CATERERS',
       "WILKEN'S FINE FOOD"], dtype=object)

In [4]:
uniquenames[-5:]


array(['CHEN EXPRESS', 'LA BOMBONIERA', 'PRET', 'TAQUERIA EL TANQUE',
       'CAFE ESPRESSO BAR'], dtype=object)

In [5]:
#Now, let's see if there are any restaurants with the string 'PANO' in them:

grep('PANO', uniquenames)

LA CANDELA ESPANOLA
PANORAMA OF MY SILENCE-HEART
GOOGLE PANORAMA
CIRCULO ESPANOL
SABOR HISPANO
PANORAMA


## The atoms
The simplest regular expressions are a sequence of atoms. An atom can be any of the following:

## single character,
a dot,
a bracket expression,
an anchor.
Single character atom
A single character atom matches itself.

## The . character atom
A dot atom matches any single character (except for a new line character \n).

Example: Using single character atoms, and the . atom, let's find all restaurant names that contain the characters AB, followed by any character (.) and then the character D:

In [6]:
grep('AB.D', uniquenames)


PABADE BAKERY & CAFE
CRAB DU JOUR XPRESS


## Bracket expression atom
A bracket expression (defined by square brackets []) defines a set of characters. matches only one single character that can be any of the characters defined in a set. Example: [ABL] matches either A, B, or L.

Now, let's use a bracket expression: We want to find restaurants that contain one of the letters A,B,C,X,Y,Z followed by a digit. We specify the set of letters as [ABCXYZ] and the set of digits as [0123456789].



In [7]:
grep('[ABCXYZ][0123456789]', uniquenames)


B66 CLUB
COTTO MARKET-GATE C30
LEGEND'S ROOM - OA3070
CARNAGIE DELI - FB8150
THE SHACK - CB8030 (CB8040)
THE SHACK - CB8030 (CB8040)
MIGHTY QUINN'S BBQ - FA8032 (CA8033)
MIGHTY QUINN'S BBQ - FA8032 (CA8033)
GARDEN MARKET EXPRESS - FA8150
FUKU - FA6090
BAR AT THE GARDEN (BA6110)
HAMBURGER, CELEBRITY CHEF, SIMPLY VEGAN FB6030 (CB6035)
HAMBURGER, CELEBRITY CHEF, SIMPLY VEGAN FB6030 (CB6035)
MIKE'S HOT HONEY - FB6060
GLUTEN FREE AT THE GARDEN/TOP DOG/HOT DOG CONCESSIONS - FB6140 (CB6142)
GLUTEN FREE AT THE GARDEN/TOP DOG/HOT DOG CONCESSIONS - FB6140 (CB6142)
FRATELLI'S - IC1001 (CC1001)
FRATELLI'S - IC1001 (CC1001)
EVENT LEVEL CLUB (DELTA SKY 360 BAR) BA5075
RAFTERS/PRIME RIB - FC6130 (CC6132)
RAFTERS/PRIME RIB - FC6130 (CC6132)
HOT DOG CONCESSION (FC6100)
THE PRIME RIB, DUNKIN DONUTS - FA8040 (CA8041)
THE PRIME RIB, DUNKIN DONUTS - FA8040 (CA8041)
FOUR SEASONS HOTEL EMPLOYEE CAFETERIA4
A1 JAMAICA BREEZE
SIX2 RESTAURANT & BAKERY
SKY55 BAR AND GRILL


## Brackets and ranges
Instead of typing long lists of characters in a bracket expression, we can use the range character: [0-9] is equivalent to [0123456789]. Similarly [A-Z] is equivalent to [ABCDEFGHIJKLMNOPQRSTUVWXYZ]. And [D-T] is equivalent to [DEFGHIJKLMNOPQRST]. (You get the idea.) You can also combine multiple ranges: [a-e1-9] is equivalent to [abcde123456789]. Finally, you can even specify to be excluded from the set using the character (^). For example, [^0-9] matches any character other than a number.

For example, let's find restaurants that contain a letter, followed by a number, and then followed by a charather that is not a number:

In [8]:
grep('[A-Z][0-9][^0-9]', uniquenames)


TWO8TWO BAR & BURGER
RIB N0.7
BT4 AISLE B
BT3 AISLE A
A1 JAMAICA BREEZE
SIX2 RESTAURANT & BAKERY
TEAMWORKON3 JUICES
O2 K-BBQ
V3NOM CAFE & LOUNGE
L8 LOUNGE
R2P
JAM1 RESTAURANT
F1RST NYC


In [9]:
# m, we do not want to get results that have a space after the number, so let's also exclude the space character:

grep('[A-Z][0-9][^0-9 ]', uniquenames)

TWO8TWO BAR & BURGER
RIB N0.7
V3NOM CAFE & LOUNGE
R2P
F1RST NYC


In [10]:
# Digit, not letter not digit not space, digit
grep('[0-9][^A-Z0-9 ][0-9]', uniquenames)

104-01 FOSTER AVENUE COFFEE SHOP(UPS)
SUNSWICK 35/35
RIB N0.7
1.50 HOT PIZZA
1.5 DAK GALBI
CHAIKHANA 7:40
$1.50 FRESH PIZZA & HOT DOG
THERAPY WINE BAR 2.0
104-01 FOSTER AVENUE COFFEE SHOP(UPS)
SUNSWICK 35/35
RIB N0.7
1.50 HOT PIZZA
1.5 DAK GALBI
CHAIKHANA 7:40
$1.50 FRESH PIZZA & HOT DOG
THERAPY WINE BAR 2.0


In [11]:

# Restaurants with five digits
grep('[0-9][0-9][0-9][0-9][0-9]', uniquenames)

STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
SUBWAY #47857
STARBUCKS COFFEE #29856
STARBUCKS COFFEE COMPANY #29897
STARBUCKS COFFEE #48170
STARBUCKS (STORE #50483)
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
MCDONALD'S #11542
MCDONALD'S #13068
MCDONALDS # 18093
MCDONALDS 14520
STARBUCKS COFFEE #50622
SUBWAY STORE #30214
CARVEL # 102224
STARBUCKS COFFEE #29719
STARBUCKS COFFEE #53473
STARBUCKS COFFEE #49450
STARBUCKS #54446
STARBUCKS COFFE #55085
TACO BELL CANTINA #034646
STARBUCKS COFFEE#54771
CARVEL 102664
STARBUCKS COFFEE #52530
TACO BELL # 35457
STARBUCKS COFFEE #58310
STARBUCKS COFFEE #56451
STARBUCKS COFFEE #57774
STARBUCKS COFFEE #57601
STARBUCKS COFFEE #58455
TACO BELL CANTINA #036012
STARBUCKS COFFEE #14090
STARBUCKS #64932
ARAMARK@ACCENTURE #19261
STARBUCKS COFFEE # 65421
STARBUCKS COFFEE #65494
STARBUCKS COFFEE #22596
BURGER KING #27012
MCDONALD'S #10407
MCDONALD'S #14106


## Anchor
Anchor atoms are special characters, used to define the location of a regex within a line.

The anchor ^ specifies the beginning of a line, the anchor $ specifies the end of a line. The anchor \b specifies the word boundary.

Example: Find restaurant names that start with the characters BAL

In [12]:
grep('^BAL', uniquenames)


BALTHAZAR RESTAURANT
BALTHAZAR BAKERY
BALLATO'S RESTAURANT
BALBOA RESTAURANT
BALADE
BALABOOSTA
BALZEM
BALVANERA
BALADE EASTERN MEDITERRANEAN
BALANCERO
BALCONY LOUNGE


In [13]:
grep('SQUARE$', uniquenames)


MADISON SQUARE
MERRION SQUARE
NORTH SQUARE
HOLIDAY INN EXPRESS NYC TIMES SQUARE
HAMPTON INN-HERALD SQUARE
RESIDENCE INN TIMES                                                                           SQUARE
THE MANHATTAN AT TIMES SQUARE
HAMPTON INN TIMES SQUARE
ELEMENT HOTEL TIME SQUARE
TACOS TIME SQUARE
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE
HOTEL RIU PLAZA NEW YORK TIMES SQUARE
MIAS BAKERY TIMES SQUARE
ICHIRAN TIMES SQUARE
SAPPHIRE GENTLEMANS CLUB TIMES SQUARE
HYATT HERALD SQUARE
PINNACLE BAGELS ON THE SQUARE
MAMAN HUDSON SQUARE
SWEETGREEN HUDSON SQUARE
AWESUM DIMSUM TIMES SQUARE
INDIA AT TIMES SQUARE
HYATT PLACE NEW YORK CITY/TIMES SQUARE
FUSHIMI AT TIMES SQUARE


In [14]:
# All restaurants that end with 4 digits
grep('[0-9][0-9][0-9][0-9]$', uniquenames)

STARBUCKS #7277
STARBUCKS COFFEE #7358
STARBUCKS #7378
GALLAGHER'S 2000
CARVEL 2848
KAFFE 1668
THE ORIGINAL VINCENT'S ESTABLISH 1904
1803
CHIPOTLE MEXICAN GRILL #2211
DUNKIN STORE #1115
CHIPTOLE MEXICAN GRILL #2407
LEGEND'S ROOM - OA3070
CARNAGIE DELI - FB8150
GARDEN MARKET EXPRESS - FA8150
FUKU - FA6090
MIKE'S HOT HONEY - FB6060
EVENT LEVEL CLUB (DELTA SKY 360 BAR) BA5075
SUITE 200, 1879
KOBEYAKI/HAMBURGER CELEBRITY CHEF - FD6010
STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
SUBWAY #47857
CARVEL 1939
CHIPOTLE MEXICAN GRILL #2308
CHIPOTLE MEXICAN GRILL #2254
PANDA EXPRESS #2679
PANDA EXPRESS #2622
CHIPOTLE MEXCIAN GRILL # 2760
STARBUCKS COFFEE #29856
CHIPOTLE MEXICAN GRILL #2834
MEXICO 2000
CHIPOTLE MEXICAN GRILL #2838
STARBUCKS COFFEE COMPANY #29897
NEW GREAT WALL 1419
CHIPOTLE MEXICAN GRILL #2570
CHIPOTLE MEXICAN GRILL #2879
STARBUCKS COFFEE #48170
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
MCD

In [15]:
 #Let's try to find restaurants containing the word MEXICO:

# Note that we capture also words like 'TULCIMEXICO' and 'MEXICOCIANA'
grep('MEXICO', uniquenames)

MEXICO LINDO RESTAURANT
PIAXTLA ES MEXICO DELI
TACOS MEXICO
SABOR A MEXICO
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TACOS Y QUESADILLAS MEXICO
TAQUITOS MEXICO RESTAURANT
MADE IN MEXICO
MEXICO EN LA PIEL
MEXICO EL SALVADOR INC
MEXICO 2000
VIVA MEXICO MEXICAN CUISINE
MEXICOCIANA
TULCIMEXICO RESTAURANT
EL RINCON DE MEXICO
VIVA MEXICO


In [16]:
# Notice that adding space is not sufficient
grep(' MEXICO ', uniquenames)

PIAXTLA ES MEXICO DELI
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TAQUITOS MEXICO RESTAURANT
VIVA MEXICO MEXICAN CUISINE


In [17]:
# The r'....' is a "raw" string, and allows us to enter
# backslash without having to "escape" the backslash.
# Otherwise Python will interpret \b as a single special
# character, and not as two characters \b that are part of the regex
grep(r'\bMEXICO\b', uniquenames)

MEXICO LINDO RESTAURANT
PIAXTLA ES MEXICO DELI
TACOS MEXICO
SABOR A MEXICO
NEW MEXICO PLACE
SABOR A MEXICO II
EL SOL DE MEXICO DELI GROCERY
LAS MARAVILLAS DE MEXICO RESTAURANT
NUEVO MEXICO MEXICAN RESTAURANT
SABOR A MEXICO TAQUERIA
TACOS Y QUESADILLAS MEXICO
TAQUITOS MEXICO RESTAURANT
MADE IN MEXICO
MEXICO EN LA PIEL
MEXICO EL SALVADOR INC
MEXICO 2000
VIVA MEXICO MEXICAN CUISINE
EL RINCON DE MEXICO
VIVA MEXICO


## Basic Patterns
a, X, 9, ....: -- ordinary characters just match themselves exactly.
. ^ \$ * + ? { [ ] \ | ( ): The meta-characters which do not match themselves because they have special meanings (more info below)
. (a period) -- matches any single character except newline '\n'
\t, \n, \r: Special characters, tab, newline, return
^ = start, $ = end -- match the start or end of the string
\: inhibit the "specialness" of a character. So, for example, use . to match a period or \ to match a slash. If you are unsure if a character has special meaning, such as '@', you can put a slash in front of it, @, to make sure it is treated just as a character.
## Shortcuts
A few of the bracket expressions that we discussed above occur very often. For this reason, we have shortcuts for them:

\d: matches the digits: [0-9].
\D: matches anything but \d: [^0-9].
\w: matches any alphanumeric character plus underscore: [A-Za-z0-9_].
\W: matches anything but \w: [^A-Za-z0-9_]
\s: matches any "whitespace" character (space, tab, newline, etc): [ \t\n\r\f\v].
\S: matches anything but \s: [^ \t\n\r\f\v] .
\b: matches the breaks between alphanumeric and non-alphanumeric characters (an empty string), the boundary between \w and \W. Useful for ensuring that what you match is actually a word.
\B: matches anything but \b. Useful for ensuring your match is in the middle of a word.
In class exercises
Write a regular expression for:

Match any character
Match the end of line
Match any digit
Find all characters that are not digits
Find all words with four letters
Find every line that starts with a digit
Find all empty lines
Find all lines with 4 characters
##Regular Expressions: Operators
Alternation |
The alternation operator | defines one or more alternatives regular expressions that need to be true for the string to match the regular expression.

For example, if we are looking for names that contain either the word GREEK or the word RUSSIAN, we issue the following command:

In [18]:
grep('GREEK|RUSSIAN|FRENCH', uniquenames)


SYMPOSIUM GREEK RESTAURANT
RUSSIAN TURKISH BATHS
RUSSIAN SAMOVAR
FRENCH ROAST
RUSSIAN VODKA ROOM
JEAN DANET FRENCH PASTRY
THE GREEK KITCHEN
RUSSIAN BATHS
GREEK ISLANDS
AVLI THE LITTLE GREEK TAVERN
THE RUSSIAN TEA ROOM
GREEK EXPRESS
SOMETHINGREEK
VILLAGE TAVERNA GREEK GRILL
GRK FRESH GREEK
AVLEE GREEK KITCHEN
GREEK XPRESS
GREEK FAMILY KITCHEN
FRENCH LOUIE
LE FRENCH DINER
DIRTY FRENCH
PARDON MY FRENCH
GREEK EATS
EXCUSE MY FRENCH
3 GREEKS GRILL
AVLI LITTLE GREEK KAFE
KUZINA THE GREEK KITCHEN
FRENCHETTE
FRENCHY COFFEE NYC
EONS GREEK FOOD FOR LIFE
CHEZ LES FRENCHIES
GREEK FROM GREECE
L'ALGEROISE FRENCH PATISSERIE
FRENCHETTE BAKERY
SIMPLY GREEK
HOW GREEK IT IS AND MORE
JEN'S FRENCH TOAST
PI GREEK BAKERIE
BROOKLYN FRENCH BAKERS


##Repetition {m,n}
A repetition operator specifies that the atom or expression immediately before the repetition may be repeated. For example, if we are looking for restaurants that contain the letter I, three to five times:

In [20]:
grep('I{3,5}', uniquenames)


KNAPP PIZZA III
BARZOLA'S RESTAURANT III
NEW WIN HING III CHINESE RESTAURANT
BAGEL EXPRESS III
LITTLE ITALY PIZZA III
ROCCO PIZZA III
MIRACALI III
EL POLLO III
CESTRAS PIZZA III
EL NUEVO VALLE III
PHO RAINBOW III
RICO POLLO III
AVOCADO SUSHI IIII
PHO BEST III
TROPILYZ III RESTAURANT
DENGJI IIII RESTAURANT
BROTHER III BAKERY
GUERRERO'S JUCE BAR III


In [21]:
#Now, let's find all the restaurants that have a name length from 50 to 55 characters:

grep('^.{50,55}$', uniquenames)

VINNY'S OF CARROLL GARDEN RESTAURANT & LUNCHEONETT
MARRIOTT MARQUIS - MAIN KITCHEN/5TH FLOOR EMPLYEE CAFE
ADVENTURES AMUSEMENTS PARK (ICE CREAM, SWEETS STAND)
ST JOHN'S UNIVERSITY LIBRARY CAFE  (ST.AUGUSTINE HALL)
CARIBBEAN CONNECTION CATERING SERVICES INC RESTAURANT
VISTA SKY LOUNGE & CATERING (SHERATON FOUR POINTS)
RESORTS WORLD CASINO GROUND LEVEL ( EMPLOYEE DINING)
THE BIG APPLE (RITZ CARLTON HOTEL EMPLOYEE CAFETERIA)
BARCLAYS LOWER SUITES TANDUAY BAR SOUTH CLUB LOUNGE
BUFFALO WILD WINGS, PEETS COFFEE & TEA, BENTO SUSHI
MANDARIN ORIENTAL NEW YORK- LOBBY LOUNGE 35TH FLOOR
ROOF AT PARK SOUTH (ENTRANCE 125 EAST 27ST, 9TH FL)
THE BISTRO AT THE COURTYARD & RESIDENCE INN BY MARRIOTT
HAMBURGER, CELEBRITY CHEF, SIMPLY VEGAN FB6030 (CB6035)
LOTTE NEW YORK PALACE - 4TH FLOOR COLD BANQUET KITCHEN
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #1
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #2
COLUMBIA UNIVERSITY BAKER ATHLETICS COMPLEX, STAND #3
FORDHAM U - SALT & SESAME AT FACULTY 

In [22]:
'''In the repetition operator {m,n}, we can skip putting the upper limit if we
want to say, "anything with m matches and above". For example, let's find all
the restaurants that have a name length 60 characters and above:'''

grep('^.{60,}$', uniquenames)

SNACK BAR (LOCATED BETWEEN A-B BETWEEN FANCY FOOD AND MASTERS)
MAIMONIDES PARK STAND # 110, TIKI BAR/MR SOFTEE/MARTY'S BURGER
MAIMONIDES PARK STAND #120, PREMIO SAUSAGE/ARANCINI/CHEESESTEAKS/MR. SOFTEE
FASHION INSTITUTE OF TECHNOLOGY DAVID DUBINSKY STUDENT CENTER
DELTA SKY CLUB (BARTENDER SERVICE TERMINAL D DELTA DEPARTURE)
RESIDENCE INN TIMES                                                                           SQUARE
INTERCONTINENTAL NEW YORK TIMES SQUARE, TODD ENGLISH THE STINGER
MARLIN BAR AT TOMMY BAHAMA AND TOMMY BAHAMA RESTAURANT AND B
BRASSIERIE 1605, BROADWAY 49 BAR & LOUNGE (MAIN KITCHEN), GRIND & CO
THE PENINSULA NEW YORK, CLEMENT/ THE GOTHAM LOUNGE/ SALON DE NING
UNION SQUARE SPORTS & ENTERTAINMENT AT THEATRE FOR A NEW AUDIENCE
FAIRFIELD INN & SUITES NEW YORK MANHATTAN FINANCIAL DISTRICT
GLUTEN FREE AT THE GARDEN/TOP DOG/HOT DOG CONCESSIONS - FB6140 (CB6142)
KINGS COUNTY DISTILLERY (GATE HOUSE ON NAVY STREET & PERRY AVE)
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHAT

##Repetition shortcuts (very common!):
* = {0,}. The * character means match the previous atom zero or more times
+ = {1,}. The + character means match the previous atom one or more times
? = {0,1}. The * character means match the previous atom zero or one times
Find all restaurants that start with one or more digits, followed by a space.



In [23]:
grep('^[0-9]+ ', uniquenames)

1 EAST 66TH STREET KITCHEN
5 BURRO CAFE
1020 BAR
810 DELI & CAFE
101 DELI
44 & X HELL'S KITCHEN
88 PALACE RESTAURANT
3 SISTERS' & SHANTA'S RESTAURANT & BAKERY
68 JAY STREET BAR
3 STAR JUICE CENTER
5 ESTRELLA BAKERY
169 BAR
809 GRILL & BAR RESTAURANT
230 FIFTH
11 STREET CAFE
234 CHINA CITY
27 SPORTS BAR & CAFE
535 MADISON CAFE
96 TEARS
86 NOODLES
95 SOUTH
1 BANANA QUEEN
2 BROS PIZZA
67 ORANGE STREET
773 LOUNGE
18 BAKERY INC
1893 SPORTS BAR
212 HISAE'S
69 BAR LOUNGE
1 STOP PATTY SHOP
1001 NIGHTS CAFE
101 CAFE
5 NAPKIN BURGER
2 BROS PIZZA, CORVO COFFEE
18 RESTAURANT
94 CORNER CAFE
888 KITCHEN
3 SHEETS SALOON
55 THOMAS CAFE
4618 BAKERY
69 PIZZERIA
120 BAY CAFE/CARGO
68 MEI MEI BAKERY
718 HOOKAH LOUNGE
54 BELOW
5 STAR VILLAGE CAFE
12 CORAZONES RESTAURANT & BAR
99 FAVOR TASTE
401 LUCKY STAR RESTAURANT
624 KAM HAI CHINESE FOOD
212 STEAKHOUSE
7 GRAMS CAFFE
99 CENTS FRESH SLICE PIZZA
19 CAFE
12 CHAIRS CAFE
750 MYRTLE DINER
3 GUYS SUSHI
99 ROGERS
310 BOWERY BAR
678 DELI
345 CANTINA
3 GREEKS GRIL

In [24]:
#Find all restaurants that start with a letter, followed by one or more digits, followed by a space.

grep('^[A-Z][0-9]+ ', uniquenames)

B66 CLUB
A1 JAMAICA BREEZE
O2 K-BBQ
L8 LOUNGE


In [25]:
# Find all restaurants
# Beggining with one or more letters // ^[A-Z]+
# followed by one or more digits // [0-9]+
# Followed by any number of charaters // .*
# and ending with BAR  // BAR$
grep('^[A-Z]+[0-9]+.*BAR$', uniquenames)

In [26]:
#Find all restaurants that start with the word STARBUCKS, followed by any number
#of characters, and then have a digit.

grep('STARBUCKS.*[0-9]+', uniquenames)

STARBUCKS #7277
STARBUCKS COFFEE #7358
STARBUCKS #7378
STARBUCKS - ENTRANCE 11 WEST 33RD ST
STARBUCKS COFFEE # 26528
STARBUCKS COFFEE #22716
STARBUCKS COFFEE #23591
STARBUCKS COFFEE #29856
STARBUCKS COFFEE COMPANY #29897
STARBUCKS COFFEE #48170
STARBUCKS (STORE #50483)
STARBUCKS COFFEE #49952
STARBUCKS #50611
STARBUCKS COFFEE # 49196
STARBUCKS #48990
STARBUCKS COFFEE #49550
STARBUCKS COFFEE #50622
STARBUCKS COFFEE #29719
STARBUCKS COFFEE #53473
STARBUCKS COFFEE #49450
STARBUCKS #54446
STARBUCKS COFFEE #823
STARBUCKS COFFE #55085
STARBUCKS COFFEE#54771
STARBUCKS COFFEE #52530
STARBUCKS COFFEE #58310
STARBUCKS COFFEE #56451
STARBUCKS COFFEE #57774
STARBUCKS COFFEE #57601
STARBUCKS COFFEE #58455
STARBUCKS COFFEE #678 HUDSON
STARBUCKS COFFEE #14090
KELLY COMMONS - KELLY'S MARKETPLACE/STARBUCKS - 1ST FLOOR
STARBUCKS #64932
STARBUCKS COFFEE # 65421
STARBUCKS COFFEE #65494
STARBUCKS COFFEE #22596


## Grouping ()
In the group operator, when a group of characters is enclosed in parentheses, the next operator applies to the whole group, not only the previous characters.

For example: Find all the restaurants that start (^) with 8 or more repetitions ({8,}) of the \w+  pattern (alphanumeric characters followed by space):

In [27]:
grep(r'^(\w+ ){8,}', uniquenames)


MARLIN BAR AT TOMMY BAHAMA AND TOMMY BAHAMA RESTAURANT AND B
HOMEWOOD SUITES BY HILTON NEW YORK MIDTOWN MANHATTAN TIMES SQUARE


##Group references
Sometimes it is handy to be able to refer to a match that was made earlier in a regex. This is done with backreferences, which refer to groups. \k is the backreference specifier, where k is a number, which refers to the k-th regular expression that was enclosed in parenthesis.

For example, find if the first character(s) of a line are the same as the last:

In [28]:
grep(r'^(.{3,}).*\1$', uniquenames)


ZOO BREWS-BRONX ZOO
ARRIBA ARRIBA
BARRACUDA BAR
KENNEDY FRIED CHICKEN
HARLEM BAR-B-Q, NABE HARLEM
PIO PIO
SEBA SEBA
RUBY'S & LITTLE RUBY'S
BARCELONA BAR
TETE-A-TETE
ANTONIO'S RESTAURANT
LOS 3 POTRILLOS
PIO-PIO
ONE AND ONE
CHOP CHOP
BOULUD SUD & EPICERIE BOULUD
GONZALEZ Y GONZALEZ
SHIMBASHI SUSHI
BARCLAYS NORTH SUITE STOLI BAR
BARCLAYS 40, 40 CLUB BAR
LOS TRES POTRILLOS
BERON BERON
PIL PIL
VIS-A-VIS
BARRY'S BOOTCAMP FUEL BAR
MANGO MANGO
TUK TUK
YIA YIA
WIKIWIKI
YUMMY YUMMY
BEAN N BEAN
PANADERIA IXTAPAN
MAKIMAKI
TOFU TOFU
SANSOOKAPSAN
BAAR BAAR
PERI PERI
KENNEDY CHICKEN
CACHAPA CACHAPA
TETE A TETE
MONOMONO
SUSHI SUSHI
LOS MARINILLOS
CIAO CIAO
SHIMA SUSHI
CHA CHA MATCHA
NICHE NICHE
ZAAB ZAAB
WAKU WAKU
TERIYAKI MONSTER
BAO BAO
SUNDAY TO SUNDAY
LOULOU
ANTOJITOS DEL MANGU RESTAURANT
PASA PASA
TOMO TOMO
BARCLAYS RISE COFFEE BAR
KENTUCKY FRIED CHICKEN
EAT OFFBEAT
BARO 1376 RESTAURANT & BAR
POPPY POPPY


In [29]:
# Or find all the restaurant names that the first 5 characters (or more) are identical to the last characters.

grep(r'^([A-Z]+)\1$', uniquenames)

NONO
MEME
WIKIWIKI
MAKIMAKI
COCO
MONOMONO
MAMA
ZIZI
LOULOU


In [30]:
#Or find the restaurants where the first character is the same as the last, and the second character is the same as the penultimate character.

grep(r'^(.)(.).*\2\1$', uniquenames)

ELEPHANT & CASTLE
RED FLAME DINER
NEW WAH KITCHEN
NEW YORKER HOTEL KITCHEN
ELI'S TABLE
TERESA'S GOURMET
MESKEREM
NICE MATIN
NEW HONG KONG KITCHEN
OH TAISHO
STILLWELL SOCCER SPORTS
NEW YORK FRIED CHICKEN
REMEDY DINER
STARLIGHT GRILL RESTAURANT & SWEETS
NEW HOP SHING KITCHEN
RADIO STAR
ENTWINE
NEW EMPIRE KITCHEN
SALINAS
NEW MING GARDEN
BURGER CLUB
NEW CHEF HONG KITCHEN
NEW PEKING KITCHEN
LA ESTRELLA TROPICAL
NEW JOHN GARDEN
POTBELLY SANDWICH SHOP
RAMINI ESPRESSO BAR
NEW LIN'S GARDEN
NEW HAPPY GARDEN
DONA ZITA MEXICAN FOOD
ANASSA TAVERNA
STARLETS
NEW SPRING GARDEN
NEW WOKS KITCHEN
NEW MING'S KITCHEN
AITA TRATTORIA
LA EXCELENCIA EMPANADAS MONUMENTAL
RED LOBSTER
RESIDENCE INN WORLD TRADE CENTER
NEW ISLAND GARDEN
HARLEM HOOKAH
NEW LIN'S KITCHEN
BAKHTAR KEBAB
NEW HAPPY JOY CHINESE KITCHEN
NEW CHINA GARDEN
NEW TEXAS FRIED CHICKEN
NEW YORK HILTON , BANQUET KITCHEN
RED GINGER
TANDIR ROKHAT
HAPISGAH
LA NACIONAL
BULL'S EYE SPORTS PUB
NEW MAY MAY KITCHEN
ADDA
NEW IMPERIAL KITCHEN
MALAI ICE CREAM
PO

In [31]:
#Find all names that have three consecutive same digits

grep(r'([0-9])\1\1', uniquenames)

GALLAGHER'S 2000
LEGENDS 000
888 KITCHEN
DUNKIN STORE #1115
333 LOUNGE
MEXICO 2000
GOOD TASTE 666
CARVEL # 102224
STARBUCKS #54446
SHAXIAN 888
STARBUCKS COFFEE #57774
CHECKERS STORE #3332
888 BK THAI
CAFE 2000
EMPIRE 111


In [32]:
#As we are going to see, these backreferences will also be of tremendous use for extraction purposes.

#### Naming groups
# The group that follows the term "DOUBLE" is named "doublewhat" and we can refer to it as \doublewhat
grep(r'DOUBLE (?P<doublewhat>\w+)', uniquenames)

DOUBLE DOWN SALOON
DOUBLE WINDSOR
DOUBLE RAINBOW
BEST DOUBLE DRAGON RESTAURANT
DOUBLE DUTCH ESPRESSO
DOUBLE TOP CHINA & TORTILLA TACO
DOUBLE DRAGON
DOUBLE ZERO
DOUBLE CRISPY BAKERY I
A & A BAKE AND DOUBLE SHOP
DOUBLE RED INC
DOUBLE TWISTER ICE CREAM AND COFFEE SHOP
DOUBLE DELIGHT ASIAN
DOUBLE CHICKEN PLEASE
DOUBLE CHINESE RESTAURANT
DOUBLE HAPPY KITCHEN


ay that you have a file with telephone numbers written in a variety of forms:

679-397-5255
2126660921
212-998-0902
888-888-2222
800-555-1211
800 555 1212
800.555.1213
(800) 555-1214
1-800-555-1215
1(800)555-1216
800-555-1212-1234
800-555-1212x1234
800-555-1212 ext. 1234
work 1-(800) 555.1212 #1234
The task is to standardize everything in the form (xxx)-xxx-xxx.

In [33]:
# prompt: standarize the above telephone numbers in (xxx) xxx-xxxx

import re

def standardize_phone_number(phone_number):
  """Standardizes a phone number to the format (xxx) xxx-xxxx.

  Args:
    phone_number: The phone number to standardize.

  Returns:
    The standardized phone number, or None if the phone number is invalid.
  """
  # Remove any non-digit characters from the phone number.
  digits = re.sub(r'\D', '', phone_number)

  # Check if the phone number has 10 or 11 digits.
  if len(digits) == 10:
    return f'({digits[:3]}) {digits[3:6]}-{digits[6:]}'
  elif len(digits) == 11 and digits[0] == '1':
    return f'({digits[1:4]}) {digits[4:7]}-{digits[7:]}'
  else:
    return None


# Example usage:
phone_numbers = [
    '679-397-5255',
    '2126660921',
    '212-998-0902',
    '888-888-2222',
    '800-555-1211',
    '800 555 1212',
    '800.555.1213',
    '(800) 555-1214',
    '1-800-555-1215',
    '1(800)555-1216',
    '800-555-1212-1234',
    '800-555-1212x1234',
    '800-555-1212 ext. 1234',
    'work 1-(800) 555.1212 #1234',
]

for phone_number in phone_numbers:
  standardized_number = standardize_phone_number(phone_number)
  if standardized_number:
    print(f'{phone_number} -> {standardized_number}')
  else:
    print(f'{phone_number} -> Invalid phone number')


679-397-5255 -> (679) 397-5255
2126660921 -> (212) 666-0921
212-998-0902 -> (212) 998-0902
888-888-2222 -> (888) 888-2222
800-555-1211 -> (800) 555-1211
800 555 1212 -> (800) 555-1212
800.555.1213 -> (800) 555-1213
(800) 555-1214 -> (800) 555-1214
1-800-555-1215 -> (800) 555-1215
1(800)555-1216 -> (800) 555-1216
800-555-1212-1234 -> Invalid phone number
800-555-1212x1234 -> Invalid phone number
800-555-1212 ext. 1234 -> Invalid phone number
work 1-(800) 555.1212 #1234 -> Invalid phone number


In [ ]:
# prompt: update the above code to use regular expressions to capture the phone number  correctly on the ones that are invalid number

```python
import re
from sqlalchemy import create_engine
import pandas as pd
# #Regular Expressions
# Regular expressions (regexes or re’s) constitute an extremely powerful, flexible and concise language for matching elements in text ranging from a few characters to complex patterns. While mastering the syntax of the regular expression language does require climbing a learning curve, this learning curve is not particularly steep, and a newcomer can find herself performing useful tasks with regular expressions almost immediately. Efforts spent learning regular expressions quickly pay off--tasks that are well suited for regular expressions abound. Indeed, regular expressions are one of the most useful computer skills, and an absolutely critical tool for data scientists.
#
# This document will present basic regular expression syntax and cover common use cases for regular expressions.
# The code below is written in Python to replicate the
# behavior of grep, the UNIX utility
# We will examine the details of how the code works in a subsequent notebook.
# For now, just execute the code, and use the function
# grep(regex_expression, name_list) as-is


def printMatches(text, regex_expression):
  BACKGROUND_YELLOW = '\x1b[43m'
  COLOR_RESET  = "\x1b[0m"
  regex= re.compile(regex_expression)
  matches = regex.finditer(text)
  for m in matches:
    highlighted  = text[:m.start()] # the string before the regex match
    highlighted += BACKGROUND_YELLOW + text[m.start():m.end()] + COLOR_RESET
    highlighted += text[m.end():] # the string after the regex match
    print(highlighted)

def grep(regex_expression, name_list):
  for line in name_list:
    printMatches(line, regex_expression)
# ## NYC Restaurant Names Data
# In the notebook, we will demonstrating the various regular expressions using the set of restaurant names from /data/uniquenames.txt.
!pip install -U -q PyMySQL sqlalchemy


conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org',
    user = 'student',
    password = 'dwdstudent2025',
    db = 'doh_restaurants',
    encoding = 'utf8mb4')

with create_engine(conn_string).connect() as mysql_conn:
  # This query returns back the restaurants in the DOH database
  sql = 'SELECT DISTINCT UPPER(DBA) AS DBA FROM restaurants WHERE DBA IS NOT NULL'
  uniquenames = pd.read_sql(sql, con=mysql_conn)
  uniquenames = uniquenames.DBA.values

print(f"Found {len(uniquenames)} unique restaurants names")
#Let's take a peek at the entries of the uniquenames list:

uniquenames[:5]
uniquenames[-5:]

#Now, let's see if there are any restaurants with the string 'PANO' in them:

grep('PANO', uniquenames)
# ## The atoms
# The simplest regular expressions are a sequence of atoms. An atom can be any of the following:
#
# ## single character,
# a dot,
# a bracket expression,
# an anchor.
# Single character atom
# A single character atom matches itself.
#
# ## The . character atom
# A dot atom matches any single character (except for a new line character \n).
#
# Example: Using single character atoms, and the . atom, let's find all restaurant names that contain the characters AB, followed by any character (.) and then the character D:
grep('AB.D', uniquenames)

# ## Bracket expression atom
# A bracket expression (defined by square brackets []) defines a set of characters. matches only one single character that can be any of the characters defined in a set. Example: [ABL] matches either A, B, or L.
#
# Now, let's use a bracket expression: We want to find restaurants that contain one of the letters A,B,C,X,Y,Z followed by a digit. We specify the set of letters as [ABCXYZ] and the set of digits as [0123456789].
#
#
grep('[ABCXYZ][0123456789]', uniquenames)

# ## Brackets and ranges
# Instead of typing long lists of characters in a bracket expression, we can use the range character: [0-9] is equivalent to [0123456789]. Similarly [A-Z] is equivalent to [ABCDEFGHIJKLMNOPQRSTUVWXYZ]. And [D-T] is equivalent to [DEFGHIJKLMNOPQRST]. (You get the idea.) You can also combine multiple ranges: [a-e1-9] is equivalent to [abcde123456789]. Finally, you can even specify to be excluded from the set using the character (^). For example, [^0-9] matches any character other than a number.
#
# For example, let's find restaurants that contain a letter, followed by a number, and then followed by a charather that is not a number:
grep('[A-Z][0-9][^0-9]', uniquenames)

# m, we do not want to get results that have a space after the number, so let's also exclude the space character:

grep('[A-Z][0-9][^0-9 ]', uniquenames)
# Digit, not letter not digit not space, digit
grep('[0-9][^A-Z0-9 ][0-9]', uniquenames)

# Restaurants with five digits
grep('[0-9][0-9][0-9][0-9][0-9]', uniquenames)
# ## Anchor
# Anchor atoms are special characters, used to define the location of a regex within a line.
#
# The anchor ^ specifies the beginning of a line, the anchor $ specifies the end of a line. The anchor \b specifies the word boundary.
#
# Example: Find restaurant names that start with the characters BAL
grep('^BAL', uniquenames)

grep('SQUARE$', uniquenames)

# All restaurants that end with 4 digits
grep('[0-9][0-9][0-9][0-9]$', uniquenames)
 #Let's try to find restaurants containing the word MEXICO:

# Note that we capture also words like 'TULCIMEXICO' and 'MEXICOCIANA'
grep('MEXICO', uniquenames)
# Notice that adding space is not sufficient
grep(' MEXICO ', uniquenames)
# The r'....' is a "raw" string, and allows us to enter
# backslash without having to "escape" the backslash.
# Otherwise Python will interpret \b as a single special
# character, and not as two characters \b that are part of the regex
grep(r'\bMEXICO\b', uniquenames)
# ## Basic Patterns
# a, X, 9, ....: -- ordinary characters just match themselves exactly.
# . ^ \$ * + ? { [ ] \ | ( ): The meta-characters which do not match themselves because they have special meanings (more info below)
# . (a period) -- matches any single character except newline '\n'
# \t, \n, \r: Special characters, tab, newline, return
# ^ = start, $ = end -- match the start or end of the string
# \: inhibit the "specialness" of a character. So, for example, use . to match a period or \ to match a slash. If you are unsure if a character has special meaning, such as '@', you can put a slash in front of it, @, to make sure it is treated just as a character.
# ## Shortcuts
# A few of the bracket expressions that we discussed above occur very often. For this reason, we have shortcuts for them:
#
# \d: matches the digits: [0-9].
# \D: matches anything but \d: [^0-9].
# \w: matches any alphanumeric character plus underscore: [A-Za-z0-9_].
# \W: matches anything but \w: [^A-Za-z0-9_]
# \s: matches any "whitespace" character (space, tab, newline, etc): [ \t\n\r\f\v].
# \S: matches anything but \s: [^ \t\n\r\f\v] .
# \b: matches the breaks between alphanumeric and non-alphanumeric characters (an empty string), the boundary between \w and \W. Useful for ensuring that what you match is actually a word.
# \B: matches anything but \b. Useful for ensuring your match is in the middle of a word.
# In class exercises
# Write a regular expression for:
#
# Match any character
# Match the end of line
# Match any digit
# Find all characters that are not digits
# Find all words with four letters
# Find every line that starts with a digit

More Advanced Regular Expressions
And the ones below get a little bit more advanced:

*?, +?: ordinarily, *, + and ? are greedy. This means they are matching the longest possible string that satisfies the regular expression. Adding the ? to any of these makes it non-greedy, instead matching the shortest possible expression.
(?: ): A non-capturing group. This works just as (), but doesn’t hold on to the matched contents.
(?<=x): Matches any string that is preceded by x (an arbitrary regular expression).